In [ ]:
'''
Overview
this notebook parses all past ufc fight stats when run, it does not include upcoming fights


scrape ufc fight stats
get all event details, name, url, date, location for all ufc events
for each event, get fight details all fights on card
parse each fight to get fight stats of both fighters
'''

In [1]:
# imports
import pandas as pd
from tqdm.notebook import tqdm_notebook

# import library
import scrape_ufc_stats_library as LIB

# import config
import yaml
config = yaml.safe_load(open('scrape_ufc_stats_config.yaml'))

# Parse Event Details
Includes:
<br>
Event
<br>
URL
<br>
Date
<br>
Location
<br>

In [2]:
# define url to parse
events_url = config['completed_events_all_url']

In [3]:
# get soup
soup = LIB.get_soup(events_url)

# parse event details
all_event_details_df = LIB.parse_event_details(soup)

# show event details
display(all_event_details_df)

# write event details to file
all_event_details_df.to_csv(config['event_details_file_name'], index=False)

,EVENT,URL,DATE,LOCATION
0,UFC Fight Night: Kattar vs. Chikadze,http://ufcstats.com/event-details/f5585e675af7...,"January 15, 2022","Las Vegas, Nevada, USA"
1,UFC Fight Night: Lewis vs. Daukaus,http://ufcstats.com/event-details/2a470ad41c22...,"December 18, 2021","Las Vegas, Nevada, USA"
2,UFC 269: Oliveira vs. Poirier,http://ufcstats.com/event-details/ef927e4fe211...,"December 11, 2021","Las Vegas, Nevada, USA"
3,UFC Fight Night: Font vs. Aldo,http://ufcstats.com/event-details/509697e08673...,"December 04, 2021","Las Vegas, Nevada, USA"
4,UFC Fight Night: Vieira vs. Tate,http://ufcstats.com/event-details/c95fbc085e17...,"November 20, 2021","Las Vegas, Nevada, USA"
...,...,...,...,...
584,UFC 6: Clash of the Titans,http://ufcstats.com/event-details/1c3f5e85b59e...,"July 14, 1995","Casper, Wyoming, USA"
585,UFC 5: The Return of the Beast,http://ufcstats.com/event-details/dedc3bb440d0...,"April 07, 1995","Charlotte, North Carolina, USA"
586,UFC 4: Revenge of the Warriors,http://ufcstats.com/event-details/b60391da771d...,"December 16, 1994","Tulsa, Oklahoma, USA"
587,UFC 3: The American Dream,http://ufcstats.com/event-details/1a49e0670dfa...,"September 09, 1994","Charlotte, North Carolina, USA"


# Parse Fight Details
Includes:
<br>
Event
<br>
Bout
<br>
URL

In [5]:
# define list of urls of fights to parse
list_of_events_urls = list(all_event_details_df['URL'])

In [6]:
# create empty df to store fight details
all_fight_details_df = pd.DataFrame(columns=config['fight_details_column_names'])

# loop through each event and parse fight details
for url in tqdm_notebook(list_of_events_urls):

    # get soup
    soup = LIB.get_soup(url)

    # parse fight links
    fight_details_df = LIB.parse_fight_details(soup)
    
    # concat fight details
    all_fight_details_df = pd.concat([all_fight_details_df, fight_details_df])

# show all fight details
display(all_fight_details_df)

# write fight details to file
all_fight_details_df.to_csv(config['fight_details_file_name'], index=False)

  0%|          | 0/589 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Parse Fight Results and Fight Stats

Fight Results Includes:
<br>
Event
<br>
Bout
<br>
Weightclass
<br>
Method
<br>
Round
<br>
Time
<br>
Time Format
<br>
Referee
<br>
Details
<br>

Fight Stats Includes:
<br>
Event
<br>
Bout
<br>
Round
<br>
Fighter
<br>
Kd
<br>
Sig.Str.
<br>
Sig.Str. %
<br>
Total Str.
<br>
Td
<br>
Td %
<br>
Sub.Att
<br>
Rev.
<br>
Ctrl
<br>
Head
<br>
Body
<br>
Leg
<br>
Distance
<br>
Clinch
<br>
Ground
<br>

In [9]:
# define list of urls of fights to parse
list_of_fight_details_urls = list(all_fight_details_df['URL'])

In [17]:
# create empty df to store fight results
all_fight_results_df = pd.DataFrame(columns=config['fight_results_column_names'])
# create empty df to store fight stats
all_fight_stats_df = pd.DataFrame(columns=config['fight_stats_column_names'])

# loop through each fight and parse fight results and stats
for url in tqdm_notebook(list_of_fight_details_urls):

    # get soup
    soup = LIB.get_soup(url)

    # parse fight results and fight stats
    fight_results_df, fight_stats_df = LIB.parse_organise_fight_results_and_stats(
        soup,
        url,
        config['fight_results_column_names'],
        config['totals_column_names'],
        config['significant_strikes_column_names']
        )

    # concat fight results
    all_fight_results_df = pd.concat([all_fight_results_df, fight_results_df])
    # concat fight stats
    all_fight_stats_df = pd.concat([all_fight_stats_df, fight_stats_df])

# show all fight results
display(all_fight_results_df)
# show all fight stats
display(all_fight_stats_df)

# write to file
all_fight_results_df.to_csv(config['fight_results_file_name'], index=False)
# write to file
all_fight_stats_df.to_csv(config['fight_stats_file_name'], index=False)

  0%|          | 0/299 [00:00<?, ?it/s]

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL
0,UFC 33: Victory in Vegas,Tito Ortiz vs. Vladimir Matyushenko,W/L,UFC Light Heavyweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),John McCarthy,Doug Crosby 43 - 50.Jeff Mullen 44 - 50.Nelson...,http://ufcstats.com/fight-details/ea64af61ed03...
0,UFC 33: Victory in Vegas,Jens Pulver vs. Dennis Hallman,W/L,UFC Lightweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Mario Yamasaki,Doug Crosby 47 - 49.Jeff Mullen 47 - 48.Cecil ...,http://ufcstats.com/fight-details/970c2beda729...
0,UFC 33: Victory in Vegas,Chuck Liddell vs. Murilo Bustamante,W/L,Light Heavyweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),John McCarthy,Tony Weeks 27 - 29.Doug Crosby 28 - 29.Jon Sch...,http://ufcstats.com/fight-details/a6a4be467d12...
0,UFC 33: Victory in Vegas,Matt Serra vs. Yves Edwards,W/L,Welterweight Bout,Decision - Majority,3,5:00,3 Rnd (5-5-5),Mario Yamasaki,Tony Weeks 29 - 29.Jeff Mullen 27 - 30.Nelson ...,http://ufcstats.com/fight-details/3b63f4fb2469...
0,UFC 33: Victory in Vegas,Dave Menne vs. Gil Castillo,W/L,UFC Middleweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),John McCarthy,Point Deducted: Illegal Knee by MenneTony Week...,http://ufcstats.com/fight-details/37cb7ce0f0b7...
...,...,...,...,...,...,...,...,...,...,...,...
0,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,W/L,Open Weight Bout,KO/TKO,1,2:50,No Time Limit,John McCarthy,toCorner Stoppage,http://ufcstats.com/fight-details/3b020d4914b4...
0,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,W/L,Open Weight Bout,Submission,1,4:52,No Time Limit,John McCarthy,Keylock From Half Guard,http://ufcstats.com/fight-details/d917c8c7461b...
0,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,W/L,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,John McCarthy,Punches to Head From GuardSubmission to Strikes,http://ufcstats.com/fight-details/ccee020be2e8...
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,W/L,Open Weight Bout,Submission,1,0:58,No Time Limit,John McCarthy,Guillotine Choke Standing,http://ufcstats.com/fight-details/4b9ae533ccb3...


,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,SUB.ATT,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND
0,UFC 33: Victory in Vegas,Tito Ortiz vs. Vladimir Matyushenko,Round 1,Tito Ortiz,0,13 of 15,86%,27 of 29,1 of 1,100%,0,0,4:29,2 of 4,3 of 3,8 of 8,0 of 0,13 of 14,0 of 1
1,UFC 33: Victory in Vegas,Tito Ortiz vs. Vladimir Matyushenko,Round 2,Tito Ortiz,0,32 of 34,94%,32 of 34,1 of 4,25%,0,0,4:15,0 of 2,2 of 2,30 of 30,1 of 2,31 of 32,0 of 0
2,UFC 33: Victory in Vegas,Tito Ortiz vs. Vladimir Matyushenko,Round 3,Tito Ortiz,0,7 of 15,46%,62 of 71,1 of 1,100%,0,0,4:30,7 of 15,0 of 0,0 of 0,1 of 6,0 of 0,6 of 9
3,UFC 33: Victory in Vegas,Tito Ortiz vs. Vladimir Matyushenko,Round 4,Tito Ortiz,0,5 of 9,55%,54 of 58,1 of 2,50%,0,0,4:37,3 of 7,2 of 2,0 of 0,0 of 1,2 of 2,3 of 6
4,UFC 33: Victory in Vegas,Tito Ortiz vs. Vladimir Matyushenko,Round 5,Tito Ortiz,0,7 of 11,63%,73 of 77,1 of 1,100%,0,0,3:43,5 of 9,1 of 1,1 of 1,3 of 6,1 of 1,3 of 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,Round 1,David Levicki,0,4 of 5,80%,95 of 102,0 of 0,---,0,0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Patrick Smith,0,1 of 1,100%,1 of 1,0 of 1,0%,1,0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Ray Wizard,0,1 of 1,100%,2 of 2,0 of 0,---,0,0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0
0,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,Round 1,Scott Morris,0,1 of 1,100%,2 of 2,1 of 1,100%,1,0,--,1 of 1,0 of 0,0 of 0,0 of 0,1 of 1,0 of 0
